In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


wrapper = DuckDuckGoSearchAPIWrapper(region="us-en", time="d", max_results=2)

#OPENAI FUNCTION
class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It taks a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


In [18]:
import yfinance as yf
import requests_cache

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Symbol of the company.Example:AAPL")

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        company = yf.Ticker(symbol, session=session)
        company_info = company.financials
        return company_info


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and pow

In [16]:

class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of the company.
    You should enter a stock symbol.
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        company = yf.Ticker(symbol, session=session)
        company_income_statement = company.income_stmt
        return company_income_statement

In [23]:
agent = initialize_agent(
    llm=llm, 
    model="gpt-4o-mini",
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool()
])

prompt = "Give me information on Cloudflare's stock, considering both its income statements and financials, and help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`


Should You Buy or Sell Cloudflare Stock? Get The Latest NET Stock Analysis, Price Target, Earnings Estimates, Headlines, and Short Interest at MarketBeat. ... Current Symbol NYSE:NET. CUSIP N/A. CIK 1477333. Web www.cloudflare.com. Phone 888-993-5273. Fax N/A. Employees 3,682. Year Founded ... posing a challenge for Cloudflare's market share. Cloudflare Q2 Earnings Highlights: Revenue Beat, EPS Beat, Guidance Shines. Cloud connectivity company Cloudflare Inc NET reported second-quarter revenue of $401.0 million, up 30% year-over-year. The revenue beat a Street consensus estimate of $394.69 million, according to da... 5 weeks ago - Benzinga. The latest Cloudflare stock prices, stock quotes, news, and NET history to help you invest and trade smarter. ... Cloudflare's market capitalization is $22.59 B by 303.68 M shares outstanding ... Real-time Cloudfl

{'input': "Give me information on Cloudflare's stock, considering both its income statements and financials, and help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Stock Analysis:\n\n#### Income Statement Overview:\n- **Net Income (2023):** -$183,949,000\n- **Net Income (2022):** -$193,381,000\n- **Net Income (2021):** -$260,309,000\n\n#### Financial Overview:\n- **Gross Profit (2023):** $989,740,000\n- **Gross Profit (2022):** $742,631,000\n- **Gross Profit (2021):** $509,292,000\n\n### Analysis:\n- Cloudflare has shown consistent growth in gross profit over the years.\n- The net income has improved from 2021 to 2022 but decreased in 2023.\n- The company has been investing heavily in research and development, which is reflected in its financials.\n- Cloudflare's financial performance indicates a focus on growth and innovation.\n\n### Investment Consideration:\n- Cloudflare's financials suggest a company with potential for growth and innovation.\n- Invest

In [ ]:
import logging

